In [ ]:
import os
import subprocess
import json
import re
import sys
import time
from faster_whisper import WhisperModel # This line was missing

# --- 설정 (사용자 지정 가능) ---
CHANNEL_URL = "https://www.youtube.com/@iamkimdal/videos"
OUTPUT_DIR = "김달_유튜브_대본"

# Whisper 모델 설정
MODEL_SIZE = "medium"
DEVICE = "cuda"     # gpu 사용할때 cuda , cpu 일때 cpu 로 변경 해줘야 함
COMPUTE_TYPE = "float16"
# -----------------------------------

def sanitize_filename(filename):
    """파일 이름으로 사용할 수 없는 문자를 제거하는 함수"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def get_all_video_urls(channel_url):
    """yt-dlp를 사용해 채널의 모든 영상 URL을 가져오는 함수"""
    print(f"채널에서 모든 영상 URL을 가져오는 중입니다...")
    try:
        command = ['yt-dlp', '--flat-playlist', '--print', 'webpage_url', channel_url]
        result = subprocess.run(
            command, capture_output=True, text=True, check=True, encoding='utf-8'
        )
        urls = [line for line in result.stdout.strip().split('\n') if line.strip() and line.startswith('http')]
        print(f"총 {len(urls)}개의 유효한 영상을 찾았습니다.")
        return urls
    except subprocess.CalledProcessError as e:
        print(f"URL 목록을 가져오는 데 실패했습니다. yt-dlp 오류:\n{e.stderr}")
        return []
    except Exception as e:
        print(f"URL 목록 처리 중 예기치 않은 오류 발생: {e}")
        return []

def get_video_info(url):
    """yt-dlp를 사용해 영상의 제목과 ID를 가져오는 함수"""
    try:
        command = ['yt-dlp', '--print', '{"title": "%(title)s", "id": "%(id)s"}', '--skip-download', url]
        process = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8')
        info = json.loads(process.stdout.strip().split('\n')[-1])
        return info['title'], info['id']
    except Exception as e:
        print(f"영상 정보({url})를 가져오는 데 실패했습니다: {e}")
        return None, None

def download_audio(url, video_id, temp_dir):
    """yt-dlp를 사용해 오디오를 다운로드하는 함수"""
    try:
        output_template = os.path.join(temp_dir, f"{video_id}.%(ext)s")
        command = [
            'yt-dlp',
            '--format', 'bestaudio/best',
            '--extract-audio',
            '--audio-format', 'm4a',
            '--output', output_template,
            '--no-mtime',
            url
        ]
        
        print(f"'{video_id}' 오디오 다운로드 중...")
        subprocess.run(command, check=True, capture_output=True, text=True, encoding='utf-8')
        
        expected_path = os.path.join(temp_dir, f"{video_id}.m4a")
        time.sleep(1)

        if not os.path.exists(expected_path):
            raise FileNotFoundError(f"다운로드 후 파일을 찾을 수 없습니다: {expected_path}")

        return expected_path
        
    except subprocess.CalledProcessError as e:
        print(f"오디오 다운로드 중 오류 발생: yt-dlp가 오류 코드를 반환했습니다.")
        print("---------- yt-dlp STDERR (오류 시) ----------")
        print(e.stderr)
        print("-----------------------------------")
        return None
    except Exception as e:
        print(f"오디오 다운로드 처리 중 예기치 않은 오류 발생: {e}")
        return None

def main():
    """메인 실행 함수"""
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()
        
    output_dir_abs = os.path.join(script_dir, OUTPUT_DIR)
    temp_dir_abs = os.path.join(script_dir, "temp_audio_files")
    
    os.makedirs(output_dir_abs, exist_ok=True)
    os.makedirs(temp_dir_abs, exist_ok=True)
    
    print("--- 자동 음성 변환 스크립트 시작 ---")
    
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True, text=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("\n[치명적 오류] FFmpeg를 찾을 수 없습니다.")
        sys.exit(1)
        
    print("Whisper 모델을 로드합니다...")
    try:
        model = WhisperModel(MODEL_SIZE, device=DEVICE, compute_type=COMPUTE_TYPE)
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        return

    video_urls = get_all_video_urls(CHANNEL_URL)
    if not video_urls:
        print("처리할 영상이 없습니다. 스크립트를 종료합니다.")
        return

    for i, url in enumerate(video_urls):
        print(f"\n--- [{i+1}/{len(video_urls)}] 영상 처리 시작: {url} ---")
        
        downloaded_audio_path = None
        try:
            video_title, video_id = get_video_info(url)
            if not video_title or not video_id:
                continue
            
            print(f"영상 제목: {video_title}")
            sanitized_title = sanitize_filename(video_title)
            output_filepath = os.path.join(output_dir_abs, f"{sanitized_title}.txt")

            if os.path.exists(output_filepath):
                print(f"'{sanitized_title}.txt' 파일이 이미 존재하므로 건너뜁니다.")
                continue
            
            downloaded_audio_path = download_audio(url, video_id, temp_dir_abs)
            if not downloaded_audio_path:
                continue

            print(f"오디오 파일 경로: {downloaded_audio_path}")

            print("음성 인식 변환 중 (GPU 사용)...")
            segments, info = model.transcribe(downloaded_audio_path, beam_size=5, language="ko")
            
            transcript = "".join([segment.text for segment in segments]).strip()
            print("음성 인식 완료.")

            with open(output_filepath, "w", encoding="utf-8") as f:
                f.write(transcript)
            print(f"대본 저장 완료: {output_filepath}")

        except Exception as e:
            print(f"영상 처리 루프 중 예기치 않은 오류 발생: {e}")
        
        finally:
            if downloaded_audio_path and os.path.exists(downloaded_audio_path):
                try:
                    os.remove(downloaded_audio_path)
                    print("임시 오디오 파일 삭제 완료.")
                except OSError as e:
                    print(f"임시 파일 삭제 실패: {e}")

    try:
        if os.path.exists(temp_dir_abs) and not os.listdir(temp_dir_abs):
            os.rmdir(temp_dir_abs)
    except OSError as e:
        print(f"임시 폴더 삭제 실패: {e}")

    print("\n--- 모든 작업이 완료되었습니다. ---")


if __name__ == "__main__":
    main()

--- 자동 음성 변환 스크립트 시작 ---
Whisper 모델을 로드합니다...
채널에서 모든 영상 URL을 가져오는 중입니다...
총 1026개의 유효한 영상을 찾았습니다.

--- [1/1026] 영상 처리 시작: https://www.youtube.com/watch?v=OyctGxxCv0g ---
영상 제목: 좋은 여자를 못 만나는 남자들의 공통점
'좋은 여자를 못 만나는 남자들의 공통점.txt' 파일이 이미 존재하므로 건너뜁니다.

--- [2/1026] 영상 처리 시작: https://www.youtube.com/watch?v=vSgU1ViQx_g ---
영상 제목: 요즘 여자들이 결혼을 해도 불행한 이유
'vSgU1ViQx_g' 오디오 다운로드 중...
오디오 파일 경로: c:\Users\mojih\OneDrive\바탕 화면\crawling\temp_audio_files\vSgU1ViQx_g.m4a
음성 인식 변환 중 (GPU 사용)...
음성 인식 완료.
대본 저장 완료: c:\Users\mojih\OneDrive\바탕 화면\crawling\김달_유튜브_대본\요즘 여자들이 결혼을 해도 불행한 이유.txt
임시 오디오 파일 삭제 완료.

--- [3/1026] 영상 처리 시작: https://www.youtube.com/watch?v=0AvQLe58qsU ---
영상 제목: 정말 심각한 요즘 여자들 인스타
'0AvQLe58qsU' 오디오 다운로드 중...
오디오 파일 경로: c:\Users\mojih\OneDrive\바탕 화면\crawling\temp_audio_files\0AvQLe58qsU.m4a
음성 인식 변환 중 (GPU 사용)...
음성 인식 완료.
대본 저장 완료: c:\Users\mojih\OneDrive\바탕 화면\crawling\김달_유튜브_대본\정말 심각한 요즘 여자들 인스타.txt
임시 오디오 파일 삭제 완료.

--- [4/1026] 영상 처리 시작: https://www.youtube.com/watch?v=TjcvE9Oyd-